# Análisis Exploratorio de Datos - Superficie comunal

Este notebook tiene como objetivo explorar una base de datos que contiene el tamaño de las comunas de Chile en formato `.csv`, preparar los datos y generar un dataset para análisis posteriores.

## Importar librerias
Inicialmente importamos las librerias necesarias para el funcionamiento del notebook.

In [1]:
import pandas as pd

## Lectura archivo: Superficie comunal
Se lee el archivo de superficie comunal utilizando pandas.<br>
El archivo se debe generar para descargar desde el siguiente [enlace](https://www.bcn.cl/siit/estadisticasterritoriales/tema?id=140).


In [2]:
# Ruta al archivo 
ruta = '../../data/raw_data/superficie_comunal.csv'

# Leer el CSV
df_superficie = pd.read_csv(ruta)

# Ver las primeras filas
df_superficie.head()

,Unidad territorial,Variable,2020
0,General Lagos (Región de Arica y Parinacota),Superficie Comunal en kilómetros Cuadrados,2244.0
1,Putre (Región de Arica y Parinacota),Superficie Comunal en kilómetros Cuadrados,5903.0
2,Arica (Región de Arica y Parinacota),Superficie Comunal en kilómetros Cuadrados,4799.0
3,Camarones (Región de Arica y Parinacota),Superficie Comunal en kilómetros Cuadrados,3927.0
4,Colchane,Superficie Comunal en kilómetros Cuadrados,4016.0


## Limpieza de dataset
Debemos renombrar las columnas para que queden con nombres acordes al estudio, se eliminan columnas inecesarias y se modifica los nombres de las comunas para eliminar el nombre de la region que viene contenida dentro de esta.

In [3]:
# Renombramos columna
df_superficie = df_superficie.rename(columns={' 2020': 'Area'})
df_superficie = df_superficie.rename(columns={'Unidad territorial': 'Comuna'})

# Eliminamos columna
df_superficie.drop(columns=[' Variable'], inplace=True)

## removemos parentesis
df_superficie['Comuna'] = df_superficie['Comuna'].str.replace(r' \([^)]+\)$', '', regex=True)

### Deteccion de datos faltantes
Buscamos datos cuyos valores no se encuentren en el dataset original, en este caso los valores faltantes vienen con un valor ` -`.

In [4]:
# Creamos una mascara donde True indica que el valor es " -" (Dato Faltante)
mascara = df_superficie['Area'] == ' -'

# Usamos la máscara para seleccionar las filas correspondientes
filas_con_guion = df_superficie[mascara]

print(filas_con_guion.shape)
filas_con_guion.head(10)


(10, 2)


,Comuna,Area
325,Guaitecas,-
326,Lago Verde,-
327,Cisnes,-
328,Aysén,-
329,Coyhaique,-
330,Río Ibáñez,-
331,Chile Chico,-
332,Cochrane,-
333,Tortel,-
334,O’higgins,-


### Ingresar valores faltantes
Una vez detectados los datos debemos colocar los valores faltantes en el dataset.<br>
Esta informacion puede encontrarse en el siguiente [enlace](https://www.bcn.cl/siit/mapoteca/comunas), el cual permite generar los reportes comunales de las comunas respectivas las cuales contienen su tamaño en m<sup>2</sup>.

In [5]:
# Ingresamos los datos faltantes
df_superficie.loc[df_superficie['Comuna'] == 'Guaitecas', 'Area'] = 620.6
df_superficie.loc[df_superficie['Comuna'] == 'Lago Verde', 'Area'] = 5422.3
df_superficie.loc[df_superficie['Comuna'] == 'Cisnes', 'Area'] = 16093.0
df_superficie.loc[df_superficie['Comuna'] == 'Aysén', 'Area'] = 29796.4
df_superficie.loc[df_superficie['Comuna'] == 'Coyhaique', 'Area'] = 7290.2

df_superficie.loc[df_superficie['Comuna'] == 'Cochrane', 'Area'] = 8599.5
df_superficie.loc[df_superficie['Comuna'] == "O’higgins", 'Area'] = 8182.5
df_superficie.loc[df_superficie['Comuna'] == 'Tortel', 'Area'] = 19710.6
df_superficie.loc[df_superficie['Comuna'] == 'Chile Chico', 'Area'] = 5737.1
df_superficie.loc[df_superficie['Comuna'] == 'Río Ibáñez', 'Area'] = 5997.2

### Modificar el tipo de dato en la columna "Area"
Finalmente cambiamos el tipo de dato de la columna *Area*, lo cual sera util para generar calculos mas adelante.

In [6]:
df_superficie['Area'] = pd.to_numeric(df_superficie['Area'])

## Lectura de archivo: Codigo de comuna
Se debe leer el archivo correspondiente al codigo de las comunas, esto para agregarlo al dataset principal, lo que permitira facilitar los cruces de datos con otras tablas al no depender del nombre de la comuna y si del codigo de esta.

In [8]:
ruta_comunas = '../../data/processed_data/codigo_comuna.csv'

df_comunas =  pd.read_csv(ruta_comunas)

df_comunas.head()

,Código comuna,Comuna
0,15101,Arica
1,15102,Camarones
2,15201,Putre
3,15202,General Lagos
4,1101,Iquique


## Verificacion nombres de comunas
Se debe verificar que los nombres de las comunas corresponda a los nombres en el archivo con el codigo comunal, esto permitira hacer la union entre ambos datasets.

In [9]:
# Comunas en minuscula para comparar
df_comunas['Comuna'] = df_comunas['Comuna'].str.lower()
df_superficie['Comuna'] = df_superficie['Comuna'].str.lower()

# Comunas faltantes en df_comunas
comunas_faltantes_comunas = set(df_superficie["Comuna"]) - set(df_comunas["Comuna"])

# Comunas faltantes en df_superficie
comunas_faltantes_df = set(df_comunas["Comuna"]) - set(df_superficie["Comuna"])

print("Comunas faltantes en df_superficie:")
print(comunas_faltantes_df)

print("\nComunas faltantes en df_comunas:")

print(comunas_faltantes_comunas)

Comunas faltantes en df_superficie:
{"o'higgins", 'antártica', 'trehuaco'}

Comunas faltantes en df_comunas:
{'o’higgins', 'treguaco'}


In [10]:
df_superficie["Comuna"] = df_superficie["Comuna"].replace({
    "o’higgins": "o'higgins",
    "treguaco": "trehuaco"
})

## Union de Datasets y exportacion a csv
Una vez comprobado que las comunas de ambos datasets coinciden, se unen en un dataset final el cual contiene los datos que se almacenaran en el dataset final.<br>
Luego se exportara a formato csv para ser utilizado en el estudio.

In [11]:
# Join en comuna
df_final = df_comunas.merge(df_superficie, on="Comuna", how="inner")

print(df_final.shape)
df_final.head()

(345, 3)


,Código comuna,Comuna,Area
0,15101,arica,4799.0
1,15102,camarones,3927.0
2,15201,putre,5903.0
3,15202,general lagos,2244.0
4,1101,iquique,2262.0


In [12]:
df_final.to_csv("../../data/processed_data/Area_territorial.csv", index=False)